# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.data.dataset_factory import FileDatasetFactory
from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from automl_data_utils import get_image_data_as_df

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'covid-pneumonia-automl'

experiment=Experiment(ws, experiment_name)

compute_name = "compute-cluster1"
vm_size = "STANDARD_NC6"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_name)
    print("Found existing cluster, using it!")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=6)
    compute_cluster = ComputeTarget.create(ws, compute_name, compute_config)
compute_cluster.wait_for_completion()

Found existing cluster, using it!


In [5]:
train_data, test_data = get_image_data_as_df(ws, './Covid19-dataset/train', './Covid19-dataset/test',"covid-pneumonia-dataset-table")

Found 251 images belonging to 3 classes.
image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (11, 130, 100, 1)
label shape: (11,)

flattened shape: (251, 13001)
Found 66 images belonging to 3 classes.
image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130,

In [6]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,12991,12992,12993,12994,12995,12996,12997,12998,12999,class
0,0.000846,0.000938,0.002184,0.001569,0.001399,0.001369,0.001276,0.001707,0.001769,0.001830,...,0.002953,0.002999,0.002891,0.002907,0.002768,0.002645,0.002614,0.002614,0.002553,0
1,0.001630,0.002045,0.001369,0.001276,0.001138,0.000754,0.000830,0.001922,0.001815,0.001599,...,0.001492,0.002584,0.002107,0.002215,0.002553,0.003091,0.003030,0.002753,0.002461,0
2,0.000154,0.000200,0.000215,0.000261,0.000338,0.000723,0.000846,0.000830,0.000800,0.000846,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2
3,0.000384,0.000477,0.000415,0.000677,0.000292,0.000277,0.000292,0.000261,0.000292,0.000292,...,0.001461,0.001369,0.000877,0.000169,0.000185,0.000154,0.000154,0.000231,0.000015,0
4,0.000000,0.000000,0.000000,0.000261,0.000323,0.000354,0.000615,0.000677,0.000769,0.000815,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1


In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

data_factory = TabularDatasetFactory.register_pandas_dataframe(train_data, target=ws.get_default_datastore(), name='CovidPneumoniaTable')

Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/17e076fa-c8e3-43be-a038-d3b525f2ee62/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# TODO: Put your automl settings here
automl_settings = {"enable_dnn":True,
                   "experiment_timeout_minutes":60,
                   "max_cores_per_iteration":-1,
                   "enable_tf":False,
                   "featurization":"off"}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task='classification',
                             primary_metric='accuracy',
                             compute_target=compute_cluster,
                             training_data=data_factory,
                             label_column_name='class',
                             **automl_settings
                             )

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
covid-pneumonia-automl,AutoML_8a093984-8240-43d3-bfd7-118739acd5d4,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_8a093984-8240-43d3-bfd7-118739acd5d4',
 'target': 'compute-cluster1',
 'status': 'Completed',
 'startTimeUtc': '2021-04-14T15:10:02.410104Z',
 'endTimeUtc': '2021-04-14T16:49:49.102022Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'compute-cluster1',
  'AMLSettingsJsonString': '{"path":null,"name":"covid-pneumonia-automl","subscription_id":"610d6e37-4747-4a20-80eb-3aad70a55f43","resource_group":"aml-quickstarts-142681","workspace_name":"quick-starts-ws-142681","region":"southcentralus","compute_target":"compute-cluster1","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
#TODO: Save the best model
best_run = remote_run.get_best_child()
best_run_metrics = best_run.get_metrics()
best_run_parameters = best_run.get_details()
best_run_properties = best_run_parameters['properties']

print("Best Run:",best_run.id)
print("Algorithm:", best_run_properties['run_algorithm'])
print("Metric:", best_run_properties['primary_metric'])
print("Best Score:",best_run_properties['score'])


Best Run: AutoML_8a093984-8240-43d3-bfd7-118739acd5d4_17
Algorithm: VotingEnsemble
Metric: accuracy
Best Score: 0.9563076923076924


In [12]:
best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_54d49c49d864719ed38df96099e3f2f69d6e90d297234221517a174003a32ab7_d.txt',
 'azureml-logs/65_job_prep-tvmps_54d49c49d864719ed38df96099e3f2f69d6e90d297234221517a174003a32ab7_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_54d49c49d864719ed38df96099e3f2f69d6e90d297234221517a174003a32ab7_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'confusion_matrix',
 'explanation/24086c11/classes.interpret.json',
 'explanation/24086c11/eval_data_viz.interpret.json',
 'explanation/24086c11/expected_values.interpret.json',
 'explanation/24086c11/features.interpret.json',
 'explanation/24086c11/global_names/0.interpret.json',
 'explanation/24086c11/global_rank/0.interpret.json',
 'explanation/24086c11/global_values/0.interpret.json',
 'explanation/24086c11/local_importance_values.interpret.json',
 'explanation/24086c11/local_importance_viz.interpret.json',
 'explan

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
print(os.getenv('AZUREML_MODEL_DIR'))

None


In [ ]:
from azureml.core import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

registered_model = best_run.register_model(model_name='covid-pneumonia-automl',
                                            model_path='./outputs/model.pkl')


# env = Environment.get(workspace, "AzureML-Minimal").clone(env_name)

# for pip_package in ["scikit-learn"]:
#     env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score_automl.py')#,
#                                     environment=env)

aciConfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               auth_enabled=True, # this flag generates API keys to secure access
                                               memory_gb=1,
                                               tags={'name': 'covid-pneumonia', 'framework': 'Keras'},
                                               description='Covid and Pneumonia Recognition using AutoML')

service_name = "my-service"
service = Model.deploy(ws, service_name, [registered_model], inference_config=inference_config, deployment_config=aciConfig)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-14 18:17:27+00:00 Creating Container Registry if not exists..
2021-04-14 18:17:43+00:00 Use the existing image.
2021-04-14 18:17:44+00:00 Generating deployment configuration..
2021-04-14 18:17:46+00:00 Submitting deployment to compute..
2021-04-14 18:17:50+00:00 Checking the status of deployment my-service.

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
input_payload = json.dumps({
    'data': test_data.tolist()
})

output = service.run(input_payload)

print(output)

TODO: In the cell below, print the logs of the web service and delete the service